## Imports

In [1]:
import tensorflow as tf
import json
import numpy as np
from google.protobuf.json_format import MessageToJson
from utils import visualization_utils as vis_util
from utils import label_map_util
import base64,os


## Variables

In [2]:
filename = 'test_data/nb_117neg_fr_2017SB_detections.record'
threshold = 0.9 # 0.9 for FasterR-CNN, 0.5 for SSD Mobilenet models, feel free to try other numbers though
filtered= True #change to False if the filter rules are not needed


#initialize
label_list =[]
positive_list=[]
filtered_list=[]
positive_flags=[]
filtered_flags=[]
detection_list =[]
num_images = 0
for example in tf.python_io.tf_record_iterator(filename):
    num_images = num_images+1
print('Total amount of images:'+str(num_images))

Total amount of images:16013


## Read Inference Record

In [3]:
for example in tf.python_io.tf_record_iterator(filename):
    result = tf.train.Example.FromString(example);
    #uncomment this if images are also packed in the reference record
    #feature = result.features.feature
    #del feature[standard_fields.TfExampleFields.image_encoded]
    
    #read the detections
    json_str = MessageToJson(result)
    dic = json.loads(json_str)['features']['feature']
    scores = dic['image/detection/score']['floatList']['value']
    label = dic['label']['bytesList']['value']
    label = base64.b64decode(label[0]).decode()
    ymax = dic['image/detection/bbox/ymax']['floatList']['value']
    ymin = dic['image/detection/bbox/ymin']['floatList']['value']
    xmax = dic['image/detection/bbox/xmax']['floatList']['value']
    xmin = dic['image/detection/bbox/xmin']['floatList']['value']
    label_list.append(label)
    positive_flags.append(0)
    filtered_flags.append(0)
    
    #filtered list
    for i in range(len(scores)):
        width = xmax[i]-xmin[i];
        height = ymax[i]-ymin[i];
        ratio = width/height
        centroid_x = (xmax[i]+xmin[i])/2
        centroid_y = (ymax[i]+ymin[i])/2
        if (scores[i]>threshold 
        #filter rules
        and width<0.5 and width>0.1 
        and height<0.5 and height>0.1 
        and ratio>0.8 and ratio<2 
        and centroid_y>0.3 and centroid_x>0.3):
            filtered_list.append(label)
            filtered_flags[-1]=1
            break;
    
    #positive list,unfiltered        
    for score in scores:
        if score > threshold:
            positive_list.append(label)
            positive_flags[-1]=1
            break;
    #break;

In [4]:
print(str(len(positive_list))+' images are detected to have higher score than '+str(threshold))
print(str(len(filtered_list))+' images left after applying the filter rules')

1098 images are detected to have higher score than 0.9
336 images left after applying the filter rules


### example of inference results of 1 image

In [5]:
dic

{'image/detection/bbox/ymax': {'floatList': {'value': [0.51754230260849,
    0.31532353162765503,
    0.44757217168807983,
    0.2272406816482544,
    0.9750005006790161]}},
 'label': {'bytesList': {'value': ['U0JfMTYwNDguanBn']}},
 'image/detection/label': {'int64List': {'value': ['1', '1', '1', '1', '1']}},
 'image/detection/score': {'floatList': {'value': [0.6873953342437744,
    3.5886085214542618e-09,
    1.0975215181119324e-10,
    2.841788783458643e-11,
    4.5363760491079486e-12]}},
 'image/detection/bbox/ymin': {'floatList': {'value': [0.4455254077911377,
    0.0,
    0.3850269913673401,
    0.0,
    0.7121134996414185]}},
 'image/detection/bbox/xmax': {'floatList': {'value': [0.40794500708580017,
    0.9847661256790161,
    0.1725223958492279,
    0.47297611832618713,
    0.8780762553215027]}},
 'image/detection/bbox/xmin': {'floatList': {'value': [0.3079005181789398,
    0.6164631247520447,
    0.1285540759563446,
    0.04908046871423721,
    0.452379047870636]}}}

## Group up the images in positive/filtered list

In [6]:
#check which list to use
if filtered==True:
    flags=filtered_flags
elif filtered==False:
    flags=positive_flags

#Initialize    
result_list=[]
status = 0   #0 means normal, 1 means anomaly
zero_count = 0
one_count = 0
start_index = 0
end_index = 0
buffer = 3   #back to normal if 3 consecutive frames are normal
minimum = 3  #filtered out if the group has less than 3 frames

for i in range(len(flags)):
    if status ==0 and flags[i] == 1:  #start counting
        status =1
        zero_count = 0
        one_count = 1
        start_index = i
    elif status==1 and flags[i]==0:  #check exit or not
        zero_count+=1
        if status==1 and zero_count>=buffer:
            status = 0
            end_index = i-buffer
            length = (end_index-start_index+1)
            if length >= minimum:
                density=one_count/(end_index-start_index+1)
                result_list.append([label_list[start_index],label_list[end_index],one_count,density])
    elif status==1 and flags[i]==1:  #continue counting
        one_count+=1
        zero_count=0

## Groups of anomaly frames in video order


In [7]:
#start_frame    end_frame    #of positives    %of positives
result_list

[['SB_119.jpg', 'SB_123.jpg', 5, 1.0],
 ['SB_729.jpg', 'SB_732.jpg', 4, 1.0],
 ['SB_741.jpg', 'SB_746.jpg', 3, 0.5],
 ['SB_752.jpg', 'SB_755.jpg', 4, 1.0],
 ['SB_1178.jpg', 'SB_1180.jpg', 2, 0.6666666666666666],
 ['SB_3677.jpg', 'SB_3686.jpg', 10, 1.0],
 ['SB_5775.jpg', 'SB_5777.jpg', 2, 0.6666666666666666],
 ['SB_5851.jpg', 'SB_5856.jpg', 6, 1.0],
 ['SB_6250.jpg', 'SB_6253.jpg', 2, 0.5],
 ['SB_6305.jpg', 'SB_6308.jpg', 3, 0.75],
 ['SB_6360.jpg', 'SB_6368.jpg', 9, 1.0],
 ['SB_6504.jpg', 'SB_6514.jpg', 11, 1.0],
 ['SB_7872.jpg', 'SB_7886.jpg', 13, 0.8666666666666667],
 ['SB_7937.jpg', 'SB_7945.jpg', 9, 1.0],
 ['SB_8767.jpg', 'SB_8769.jpg', 2, 0.6666666666666666],
 ['SB_8810.jpg', 'SB_8814.jpg', 4, 0.8],
 ['SB_8823.jpg', 'SB_8825.jpg', 3, 1.0],
 ['SB_8836.jpg', 'SB_8839.jpg', 2, 0.5],
 ['SB_8863.jpg', 'SB_8865.jpg', 3, 1.0],
 ['SB_8876.jpg', 'SB_8899.jpg', 19, 0.7916666666666666],
 ['SB_8907.jpg', 'SB_8911.jpg', 3, 0.6],
 ['SB_9284.jpg', 'SB_9286.jpg', 3, 1.0],
 ['SB_9371.jpg', 'SB_9374.

## Ranked Groups

In [8]:
#rank by num of positives and %of positives
result_list_sorted = sorted(result_list,key = lambda x: [float(x[3]),int(x[2])],reverse=True)
result_list_sorted

[['SB_6504.jpg', 'SB_6514.jpg', 11, 1.0],
 ['SB_3677.jpg', 'SB_3686.jpg', 10, 1.0],
 ['SB_10443.jpg', 'SB_10452.jpg', 10, 1.0],
 ['SB_12214.jpg', 'SB_12223.jpg', 10, 1.0],
 ['SB_6360.jpg', 'SB_6368.jpg', 9, 1.0],
 ['SB_7937.jpg', 'SB_7945.jpg', 9, 1.0],
 ['SB_10398.jpg', 'SB_10405.jpg', 8, 1.0],
 ['SB_5851.jpg', 'SB_5856.jpg', 6, 1.0],
 ['SB_12574.jpg', 'SB_12579.jpg', 6, 1.0],
 ['SB_119.jpg', 'SB_123.jpg', 5, 1.0],
 ['SB_729.jpg', 'SB_732.jpg', 4, 1.0],
 ['SB_752.jpg', 'SB_755.jpg', 4, 1.0],
 ['SB_10717.jpg', 'SB_10720.jpg', 4, 1.0],
 ['SB_8823.jpg', 'SB_8825.jpg', 3, 1.0],
 ['SB_8863.jpg', 'SB_8865.jpg', 3, 1.0],
 ['SB_9284.jpg', 'SB_9286.jpg', 3, 1.0],
 ['SB_9413.jpg', 'SB_9433.jpg', 19, 0.9047619047619048],
 ['SB_12724.jpg', 'SB_12733.jpg', 9, 0.9],
 ['SB_9562.jpg', 'SB_9579.jpg', 16, 0.8888888888888888],
 ['SB_7872.jpg', 'SB_7886.jpg', 13, 0.8666666666666667],
 ['SB_12554.jpg', 'SB_12560.jpg', 6, 0.8571428571428571],
 ['SB_12627.jpg', 'SB_12633.jpg', 6, 0.8571428571428571],
 ['SB_

## Precision & Recall on 2017 Winter SouthBound video

In [9]:
k=len(result_list_sorted)
#uncomment this to see @10 measures
#k=10

#these are the frames where each anomaly can be seen clearly
#consider true positive if these observed anomaly is presented in one of the groups
anomaly_list = sorted([12725,12628,12574,12552,12476,12215,10447,10399,7940,7879,6907,6505,6362,4160,3680,730])
anomaly_back_list = sorted([5388,5775,5826,5852,5892,10483,10544])
detected_flags = [0]*len(anomaly_list)
detected_back_flags = [0]*len(anomaly_back_list)

for r in range(k):
    result = result_list_sorted[r]
    first = label_list.index(result[0])
    last = label_list.index(result[1])
    #check each frame in anomaly list
    for n in range(len(anomaly_list)):
        #find if this frame is in the group
        if detected_flags[n]==0:
            string = 'SB_'+str(anomaly_list[n])+'.jpg'
            index = label_list.index(string)
            # consider +- 2 frames near the anomaly
            num = index-2
            count=0
            # consider detected if having 2 or more frames overlapping
            for i in range(5):
                if num+i in range(first,last):
                    count = count+1
                if count==2:
                    detected_flags[n]=1
                    break
    #check each frame in back of anomaly list                
    for n in range(len(anomaly_back_list)):
        if detected_back_flags[n]==0:
            string = 'SB_'+str(anomaly_back_list[n])+'.jpg'
            index = label_list.index(string)
            num = index-2
            count=0
            for i in range(5):
                if num+i in range(first,last):
                    count = count+1
                if count==2:
                    detected_back_flags[n]=1
                    break
print('Precision@'+str(k)+' ='+str(sum(detected_flags)/k))
print('Recall@'+str(k)+' ='+str(sum(detected_flags)/len(anomaly_list)))

print('\nback of anomalies included in precision')
print('Precision@'+str(k)+' ='+str((sum(detected_flags)+sum(detected_back_flags))/k))
print('Recall@'+str(k)+' ='+str(sum(detected_flags)/len(anomaly_list)))
print('back_Recall@'+str(k)+' ='+str(sum(detected_back_flags)/len(anomaly_back_list)))

Precision@39 =0.3076923076923077
Recall@39 =0.75

back of anomalies included in precision
Precision@39 =0.38461538461538464
Recall@39 =0.75
back_Recall@39 =0.42857142857142855


In [10]:
for idx,frame in enumerate(anomaly_list):
    if detected_flags[idx]==1:
        print('frame '+str(frame)+' -- detected')
    else:
        print('frame '+str(frame)+' -- miss')

frame 730 -- detected
frame 3680 -- detected
frame 4160 -- miss
frame 6362 -- detected
frame 6505 -- detected
frame 6907 -- miss
frame 7879 -- detected
frame 7940 -- detected
frame 10399 -- detected
frame 10447 -- detected
frame 12215 -- detected
frame 12476 -- miss
frame 12552 -- miss
frame 12574 -- detected
frame 12628 -- detected
frame 12725 -- detected
